In [68]:
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as mp
pCTR_train = pd.read_pickle('pCTR_validation')
rtb_val = pd.read_csv('validation.csv')

# Linear Bidding using Adaboost for CTR predicion

In [71]:
ada_val = pd.read_pickle('ada_boost_val')
ada_val['pCTR'].median()

0.4909275735338956

In [470]:
# Make a table with the bidprice with premium only for impressions with  65%+ of click probability, otherwise bid 0.
# inear_bid = avgPayprice/1000 * (2*stDev)
df = pd.read_pickle('linear_bid')
df2 = df.drop(columns = ['bidprice', 'linear_bid', 'bidid'] )


#premium based on taking all impressions with click prob > 0.65

df2['bidPrice'] = 21
df2['impression'] = 0
#pCTR range goes form 0.47 to 0.57
df2['pCTR'] = ada_val['pCTR']

#premium is exponential of pCTR
df2['premium'] = 2* np.exp(df2['pCTR'])

#base_bid = validation_payprice.mean()/1000 + (2*std )
df2 ['base_bid'] = df ['linear_bid']

#update bidPrice column with premium
df2.loc[df2.pCTR>=0.5, 'bidPrice'] = np.exp(1.64*df2['premium'])
df2.loc[df2.pCTR<0.5, 'bidPrice'] = (np.exp(1.19*df2['premium']))
#new way of updating bidprice exponentially
#df2 ['bidPrice'] = np.exp(np.exp(df2['premium']))


#update impression column 
df2['impression'] = np.where ((df2['bidPrice'] >= df2['payprice']), 1, 0)
#df1['impression'].count()


#print it


#save the ada boost df in a pickle
df2.to_pickle('adaBoost_df')
adaBoost_df = pd.read_pickle('adaBoost_df')

df2.head()

,pCTR,click,payprice,bidPrice,impression,premium,base_bid
0,0.491136,0,23,48.878033,1,3.268343,81.996435
1,0.490874,0,75,48.828252,0,3.267487,81.996435
2,0.491270,0,65,48.903521,0,3.268781,81.996435
3,0.494251,0,6,49.474727,1,3.278539,81.996435
4,0.482570,0,5,47.283200,1,3.240466,81.996435


NOW TAKE INTO ACCOUNT FEATURES

In [482]:
#drop unnecessary features in rtb_val:

rtb_val_features = rtb_val.drop(columns=['click', 'payprice', 'bidprice','bidid','userid','useragent','IP','adexchange','domain','url','urlid','slotid','slotwidth','slotheight','slotvisibility','slotformat','slotprice','creative','keypage','usertag'])

df2_features = pd.merge(df2, rtb_val_features, left_index=True, right_index=True )

df2_features['base_bid'].max()
df2_features.head()
df2_features['bidPrice'].mean()
#df2_features.head()
df2_features['pCTR'].max()

0.5776061990131456

EXPONENTIAL RELATIONSHIP TO pCTR

In [472]:
#hours 17 and 18 (ctr >0.001) and most clicks
#weekday 2(ctr>0.00096) and 3(ctr>0.00083) or 5. 
# advertiser 1458 , 3358, 3386 (not much relevant because higher ctr is around 0.00078)
#region 146, 325, 333, 27, 106 , 368, 15, 201, 298, 124, 394, 393, 0, 253, 134, 238, 1 have 0.0011<ctr < 0.0015 that is above the opne of our model

In [473]:
#if ((df2_features['weekday'].values[i]) == 2 or 3 ):
            #if ((df2_features['hour'].values[i]) == 17 or 18):
                #if ((df2_features['region'].values[i]) == 325 or 368 or 324 or 393 or 146 or 325 or 333 or 27 or 106 or 368 or 15 or 201 or 298 or 124 or 394 or 393 or 0 or 253 or 134 or 238 or 1):

In [475]:

#run the algorithm to sepend the budget
#if ((df2_features['weekday'].values[i]) == 2):
budget = 6250
impressions = 0
bids = 0
clicks = 0
for i in range(0, len(df2_features)):
    if budget > 0 and budget > (df2_features['payprice'].values[i]/1000):
        impressions += 1
        #if ((df2_features['weekday'].values[i]) == 2 or 3):
            #if ((df2_features['region'].values[i]) == 325 or 368 or 394):
            # if ((df2_features['hour'].values[i]) == 17 or 18):
        if (df2_features['bidPrice'].values[i]) > (df2_features['payprice'].values[i]):
            budget = budget - (df2_features['payprice'].values[i]/1000)
            bids += 1
            clicks += df2_features['click'].values[i] 

    else:
        print("Budget is spent!")
        break
consumedBudget = 6250 - budget
ctr = (clicks/bids)*100   
cpm = ((consumedBudget)/bids)*1000
cpc = (consumedBudget)/clicks 
#Output for the alrogithm
print("Output")
print("Number of Bids:",bids)
print("Number of Impressions:",impressions)
print("Number of Clicks:", clicks)
print("CTR is:",ctr, "%")
print("CPM is:",cpm)
print("CPC is:",cpc)
print("Budget consumed is:", 6250-budget, "Yen")


Output
Number of Bids: 134667
Number of Impressions: 303925
Number of Clicks: 128
CTR is: 0.09504926968002554 %
CPM is: 46.039623664312444
CPC is: 48.43764062501535
Budget consumed is: 6200.0180000019645 Yen


# Liear Bidding using Decision Tree for CTR prediction

In [371]:
decision_tree_val = pd.read_pickle('decision_tree_val')
decision_tree_val.head()

,pCTR,click
0,0.357459,0
1,0.330309,0
2,0.357459,0
3,0.357459,0
4,0.330309,0


In [491]:
# Make a table with the bidprice with premium only for impressions with  65%+ of click probability, otherwise bid 0.
# inear_bid = avgPayprice/1000 * (2*stDev)
df = pd.read_pickle('linear_bid')
df3 = df.drop(columns = ['bidprice', 'linear_bid', 'bidid'] )




df3['bidPrice'] = 0
df3['impression'] = 0
#pCTR range goes form 0.47 to 0.57
df3['pCTR'] = decision_tree_val['pCTR']

#premium is twice the exponential to pCTR
df3['premium'] = 2* np.exp(df2['pCTR'])

#base_bid = validation_payprice.mean()/1000 + (2*std )
df3 ['base_bid'] = df ['linear_bid']

#bid price growth is exponentially proportional to premium 
df3.loc[df3.pCTR>=0.50, 'bidPrice'] = np.exp(1.65*df3['premium'])
df3.loc[df3.pCTR<0.50, 'bidPrice'] = (np.exp(1.15*df3['premium']))
#new way of updating bidprice exponentially
#df2 ['bidPrice'] = np.exp(np.exp(df2['premium']))


#update impression column 
df3['impression'] = np.where ((df3['bidPrice'] >= df3['payprice']), 1, 0)
#df1['impression'].count()


#print it


#save the ada boost df in a pickle
df3.to_pickle('adaBoost_df')
adaBoost_df = pd.read_pickle('adaBoost_df')

df3.head()
df3['bidPrice'].max()
df3['pCTR'].min()
print(87.3-33)


54.3


In [476]:
#run the algorithm to sepend the budget

budget = 6250
impressions = 0
bids = 0
clicks = 0
for i in range(0, len(df3)):
    if budget > 0 and budget > (df3['payprice'].values[i]/1000):
        impressions += 1
        if (df3['bidPrice'].values[i]) > (df3['payprice'].values[i]):
            budget = budget - (df3['payprice'].values[i]/1000)
            bids += 1
            clicks += df3['click'].values[i] 
    else:
        print("Budget is spent!")
        break
consumedBudget = 6250 - budget
ctr = (clicks/bids)*100  
cpm = ((consumedBudget)/bids)*1000
cpc = (consumedBudget)/clicks 
#Output for the alrogithm
print("Output")
print("Number of Bids:",bids)
print("Number of Impressions:",impressions)
print("Number of Clicks:", clicks)
print("CTR is:",ctr, "%")
print("CPM is:",cpm)
print("CPC is:",cpc)
print("Budget left is:", 6250-budget, "Yen")

Budget is spent!
Output
Number of Bids: 131209
Number of Impressions: 284795
Number of Clicks: 127
CTR is: 0.09679214078302555 %
CPM is: 47.63300535787908
CPC is: 49.21164566930674
Budget left is: 6249.879000001956 Yen


# VOTING 

In [350]:
voting_val = pd.read_pickle('voting_clf_pCTR_val')
voting_val.head()

,pCTR,click
0,0.438947,0
1,0.450371,0
2,0.415957,0
3,0.438027,0
4,0.394790,0


In [537]:
# inear_bid = avgPayprice/1000 * (2*stDev)
df = pd.read_pickle('linear_bid')
df1 = df.drop(columns = ['bidprice', 'linear_bid', 'bidid'] )




df1['bidPrice'] = 0
df1['impression'] = 0
#pCTR range goes form 0.47 to 0.57
df1['pCTR'] = voting_val['pCTR']

#premium is twice the exponential to pCTR
df1['premium'] = 2* np.exp(df1['pCTR'])


#bid price growth is exponentially proportional to premium 
df1.loc[df1.pCTR>=0.50, 'bidPrice'] = np.exp(1.59*df1['premium'])
df1.loc[df1.pCTR<0.50, 'bidPrice'] = (np.exp(1.13*df1['premium']))
#new way of updating bidprice exponentially
#df2 ['bidPrice'] = np.exp(np.exp(df2['premium']))


#update impression column 
df1['impression'] = np.where ((df1['bidPrice'] >= df1['payprice']), 1, 0)
#df1['impression'].count()


#print it


#save the ada boost df in a pickle
#df1.to_pickle('adaBoost_df')
#adaBoost_df = pd.read_pickle('adaBoost_df')

df1.head()
df1['bidPrice'].min()
print (84.33-26)


58.33


In [530]:
#run the algorithm to sepend the budget

budget = 6250
impressions = 0
bids = 0
clicks = 0
for i in range(0, len(df1)):
    if budget > 0 and budget > (df1['payprice'].values[i]/1000):
        impressions += 1
        if (df1['bidPrice'].values[i]) > (df1['payprice'].values[i]):
            budget = budget - (df1['payprice'].values[i]/1000)
            bids += 1
            clicks += df1['click'].values[i] 
    else:
        print("Budget is spent!")
        break
consumedBudget = 6250 - budget
ctr = (clicks/bids)*100  
cpm = ((consumedBudget)/bids)*1000
cpc = (consumedBudget)/clicks 
#Output for the alrogithm
print("Output")
print("Number of Bids:",bids)
print("Number of Impressions:",impressions)
print("Number of Clicks:", clicks)
print("CTR is:",ctr, "%")
print("CPM is:",cpm)
print("CPC is:",cpc)
print ("Budget left is:", 6250-budget, "Yen")

Output
Number of Bids: 120769
Number of Impressions: 303925
Number of Clicks: 136
CTR is: 0.11261168014970728 %
CPM is: 50.97474517468983
CPC is: 45.26594852942732
Budget left is: 6156.169000002115 Yen


# 